In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from recpack.preprocessing.preprocessors import DataFramePreprocessor
from recpack.preprocessing.filters import MinItemsPerUser, MinUsersPerItem
from recpack.algorithms import TARSItemKNN, ItemKNN
from recpack.scenarios import Timed
from recpack.matrix import InteractionMatrix

# import utils file from previous lecture
import sys
sys.path.append('../lecture4')
from utils import DATA_PATH, customer_hex_id_to_int
from evaluation import apk

# This file builds on the code in https://github.com/radekosmulski/personalized_fashion_recs/blob/main/03a_Basic_Model_Local_Validation.ipynb

In [2]:
transactions = pd.read_parquet(f'{DATA_PATH}/transactions_train.parquet')
customers = pd.read_parquet(f'{DATA_PATH}/customers.parquet')
articles = pd.read_parquet(f'{DATA_PATH}/articles.parquet')

In [3]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [4]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 7.87 s, sys: 200 ms, total: 8.07 s
Wall time: 8.06 s


In [5]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 239 ms, sys: 18.5 ms, total: 258 ms
Wall time: 257 ms


In [6]:
candidates_last_purchase = transactions.copy()

In [7]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 6.94 s, sys: 72.1 ms, total: 7.02 s
Wall time: 7.02 s


### Bestsellers candidates

In [8]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [9]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [10]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [11]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [12]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [13]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [14]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)
candidates_bestsellers_test_week.head()

,t_dat,customer_id,sales_channel_id,week,article_id,bestseller_rank,price
0,2020-07-15,272412481300040,1,105,924243001,1,0.041535
1,2020-07-15,272412481300040,1,105,924243002,2,0.041877
2,2020-07-15,272412481300040,1,105,918522001,3,0.041435
3,2020-07-15,272412481300040,1,105,923758001,4,0.033462
4,2020-07-15,272412481300040,1,105,866731001,5,0.025015


In [15]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)
candidates_bestsellers.head()

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-07-22,200292573348128,2,96,760084003,0.025094
1,2020-07-22,200292573348128,2,96,866731001,0.024919
2,2020-07-22,200292573348128,2,96,600886001,0.022980
3,2020-07-22,200292573348128,2,96,706016001,0.033197
4,2020-07-22,200292573348128,2,96,372860002,0.013193


### K most similar items

In [16]:
proc = DataFramePreprocessor(item_ix='article_id', user_ix='customer_id', timestamp_ix='week')
proc.add_filter(MinUsersPerItem(10, item_ix='article_id', user_ix='customer_id'))
proc.add_filter(MinItemsPerUser(10, item_ix='article_id', user_ix='customer_id'))

transaction_matrix = proc.process(transactions)

  0%|          | 0/1228106 [00:00<?, ?it/s]

  0%|          | 0/1228106 [00:00<?, ?it/s]

In [17]:
knn = TARSItemKNN(K=580, fit_decay=0.1, predict_decay=1/3, similarity='cosine')
knn.fit(transaction_matrix)

2022-11-27 12:37:49,033 - base - recpack - INFO - Fitting TARSItemKNN complete - Took 9.38s


TARSItemKNN(K=580, fit_decay=0.1, predict_decay=0.3333333333333333)

In [18]:
prediction_matrix = knn.predict(transaction_matrix)

In [19]:
def not_bought_by_user(interaction_matrix: csr_matrix) -> csr_matrix:
    """given a csr_matrix, set all values > 0 to 0, and all values == 0 to 1
    """
    return csr_matrix(np.ones(interaction_matrix.shape) - (interaction_matrix >= 1), dtype=int)

In [20]:
# only keep predictions for items that the user has not bought yet
not_bought_matrix = not_bought_by_user(transaction_matrix.values)

In [21]:
similar_unbought_items_matrix = prediction_matrix.multiply(not_bought_matrix)

In [22]:
def top_n_idx_sparse(matrix: csr_matrix, n: int):
    """Return index of top n values in each row of a sparse matrix.
    source: https://stackoverflow.com/questions/49207275/finding-the-top-n-values-in-a-row-of-a-scipy-sparse-matrix
    """
    top_n_idx = []
    for le, ri in zip(matrix.indptr[:-1], matrix.indptr[1:]):
        n_row_pick = min(n, ri - le)
        top_n_idx.append(matrix.indices[le + np.argpartition(matrix.data[le:ri], -n_row_pick)[-n_row_pick:]].tolist())
    
    # Get the values corresponding to the indices
    top_n_values = []
    for row_idx, col_idxs in enumerate(top_n_idx):
        top_n_values.append(matrix[row_idx, col_idxs].toarray().tolist()[0])
        assert(len(top_n_values[row_idx]) == len(top_n_idx[row_idx]))
    return top_n_idx, top_n_values

def get_top_k_similar_articles_per_user(prediction_matrix: csr_matrix, interaction_matrix: InteractionMatrix, k: int) -> pd.DataFrame:
    """given a prediction matrix and a transaction matrix, return a dataframe with the top k similar articles per user
    """
    # use interaction_matrix._df to map back to original customer and article ids
    uid_cid_map = interaction_matrix._df[["uid", "customer_id"]].drop_duplicates().set_index("uid").to_dict()["customer_id"]
    iid_aid_map = interaction_matrix._df[["iid", "article_id"]].drop_duplicates().set_index("iid").to_dict()["article_id"]

    # get column indices of top k articles per user
    top_k_idx, top_k_values = top_n_idx_sparse(prediction_matrix, k)

    similar_customers = []
    similar_articles = []
    similarity_scores = []

    for i, row in enumerate(top_k_idx):
        user_predictions = [iid_aid_map[iid] for iid in row]
        similar_customers.extend([uid_cid_map[i]] * len(user_predictions))
        similar_articles.extend(user_predictions)
        similarity_scores.extend(top_k_values[i])

        
    assert len(similar_customers) == len(similar_articles) == len(similarity_scores), "lengths of lists should be equal"
    return pd.DataFrame({"customer_id": similar_customers, "article_id": similar_articles, "similarity": similarity_scores})

In [23]:
similar_items = get_top_k_similar_articles_per_user(prediction_matrix, transaction_matrix, k=12)

In [24]:
test_set_transactions.head()

,t_dat,customer_id,sales_channel_id,week
0,2020-07-15,272412481300040,1,105
1,2020-07-15,1456826891333599,1,105
2,2020-07-15,2133687643102426,2,105
3,2020-07-15,6010692573790711,1,105
4,2020-07-15,6171059100114610,2,105


In [25]:
candidates_similar_items = pd.merge(
    similar_items,
    test_set_transactions,
    on='customer_id',
    how='left'
)

candidates_similar_items.drop(columns='similarity', inplace=True)
candidates_similar_items.head()

,customer_id,article_id,t_dat,sales_channel_id,week
0,272412481300040,926500001,2020-07-15,1,105
1,272412481300040,921906002,2020-07-15,1,105
2,272412481300040,921906001,2020-07-15,1,105
3,272412481300040,896169005,2020-07-15,1,105
4,272412481300040,788647002,2020-07-15,1,105


In [26]:
# get the last price of all article_id's
last_price = transactions \
    .groupby(['article_id', 'week']) \
    .price \
    .last() \
    .reset_index() \
    .groupby('article_id') \
    .price \
    .last() \
    .reset_index()
    
last_price.head()

,article_id,price
0,108775015,0.002068
1,108775044,0.008458
2,110065001,0.006763
3,110065002,0.005186
4,110065011,0.011847


In [27]:
candidates_similar_items = pd.merge(
    candidates_similar_items,
    last_price,
    on='article_id',
    how='left'
)

candidates_similar_items.head()

,customer_id,article_id,t_dat,sales_channel_id,week,price
0,272412481300040,926500001,2020-07-15,1,105,0.059305
1,272412481300040,921906002,2020-07-15,1,105,0.033881
2,272412481300040,921906001,2020-07-15,1,105,0.033881
3,272412481300040,896169005,2020-07-15,1,105,0.050831
4,272412481300040,788647002,2020-07-15,1,105,0.013542


### Time-weighted popularity candidates

In [28]:
popularity = transactions\
    .groupby(['article_id', 'week']).size().reset_index(name='weekly_purchase_count')\

popularity.head()

,article_id,week,weekly_purchase_count
0,108775015,95,2
1,108775015,96,1
2,108775044,95,3
3,108775044,96,8
4,108775044,97,6


In [29]:
weekly_popularity = []
def func(row):
    weeks_before = popularity[(row.article_id == popularity.article_id) & (row.week > popularity.week)]
    # get last row of weeks_before
    previous_week_popularity = 0
    if weeks_before.shape[0] > 0:
        previous_week_popularity = weekly_popularity[-1]
    return previous_week_popularity / 2.0 + float(row.weekly_purchase_count)

In [30]:
# iterate over all rows
for i, row in popularity.iterrows():
    weekly_popularity.append(func(row))
popularity['weekly_popularity'] = weekly_popularity

In [31]:
popular_articles_per_week = popularity.sort_values(['week', 'weekly_popularity'], ascending=False)\
    .groupby('week').head(20).reset_index(drop=True)

In [32]:
popular_articles_previous_week = pd.merge(popular_articles_per_week, mean_price, on=['week', 'article_id']).reset_index(drop=True)
# make a new column to rank the weekly_popularity
popular_articles_previous_week['last_week_popularity_rank'] = popular_articles_previous_week.groupby('week')['weekly_popularity'].rank(ascending=False).astype(np.int32)
popular_articles_previous_week.week += 1

In [33]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [34]:
candidates_most_popular = pd.merge(
    unique_transactions,
    popular_articles_previous_week,
    on='week',
)

In [35]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [36]:
candidates_most_popular_test_week = pd.merge(
    test_set_transactions,
    popular_articles_previous_week,
    on='week'
)
candidates_most_popular_test_week.head()

,t_dat,customer_id,sales_channel_id,week,article_id,weekly_purchase_count,weekly_popularity,price,last_week_popularity_rank
0,2020-07-15,272412481300040,1,105,924243001,852,1236.875000,0.041535,1
1,2020-07-15,272412481300040,1,105,909370001,537,1190.500000,0.032640,2
2,2020-07-15,272412481300040,1,105,751471001,526,1173.253906,0.033423,3
3,2020-07-15,272412481300040,1,105,918522001,609,1064.500000,0.041435,4
4,2020-07-15,272412481300040,1,105,448509014,490,1004.210938,0.041630,5


In [37]:
candidates_most_popular = pd.concat([candidates_most_popular, candidates_most_popular_test_week])
candidates_most_popular.drop(columns=['weekly_purchase_count', 'weekly_popularity', 'last_week_popularity_rank'], inplace=True)
candidates_most_popular.head()

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-07-22,200292573348128,2,96,760084003,0.025094
1,2020-07-22,200292573348128,2,96,866731001,0.024919
2,2020-07-22,200292573348128,2,96,600886001,0.022980
3,2020-07-22,200292573348128,2,96,706016001,0.033197
4,2020-07-22,200292573348128,2,96,372860002,0.013193


# Combining transactions and candidates / negative examples

In [38]:
transactions['purchased'] = 1

In [39]:
data = pd.concat([transactions, candidates_last_purchase, candidates_similar_items])
data.purchased.fillna(0, inplace=True)

data.purchased.mean()

0.43770432693069344

In [40]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

### Add bestseller information

In [41]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [42]:
data = data[data.week != data.week.min()]

### Add item similarity information

In [43]:
data = pd.merge(
    data, 
    similar_items[['customer_id', 'article_id', 'similarity']], 
    on=['customer_id', 'article_id'], 
    how='left'
)

### Add item popularity information

In [44]:
data = pd.merge(
    data,
    popular_articles_previous_week[['week', 'article_id', 'weekly_purchase_count', 'weekly_popularity', 'last_week_popularity_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [45]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

data['weekly_purchase_count'].fillna(0, inplace=True)
data['weekly_popularity'].fillna(0, inplace=True)
data['similarity'].fillna(data['purchased'], inplace=True)
data['bestseller_rank'].fillna(data.bestseller_rank.max() + 1, inplace=True)
data['last_week_popularity_rank'].fillna(data.last_week_popularity_rank.max() + 1, inplace=True)
data.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,similarity,weekly_purchase_count,...,section_name,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,2020-07-22,200292573348128,880777001,0.025407,2,96,1.0,13.0,1.0,0.0,...,27,1018,12,9213,1,1,0,1,25,63947
1,2020-07-22,200292573348128,784332002,0.025407,2,96,1.0,13.0,1.0,0.0,...,12,1005,0,7303,1,1,0,1,25,63947
2,2020-07-22,200292573348128,827968001,0.016932,2,96,1.0,10.0,1.0,374.0,...,30,1002,2,1227,1,1,0,1,25,63947
3,2020-07-22,200292573348128,599580086,0.011847,2,96,1.0,13.0,1.0,0.0,...,22,1018,12,52,1,1,0,1,25,63947
4,2020-07-22,248294615847351,720504008,0.031458,1,96,1.0,13.0,1.0,0.0,...,46,1016,11,95,-1,-1,0,0,46,8666


### preprocessing

In [46]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()
columns_to_scale = ['product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'last_week_popularity_rank', 'similarity']
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

In [47]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [48]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [49]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [50]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'last_week_popularity_rank', 'similarity']

In [51]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 57.6 ms, sys: 144 ms, total: 201 ms
Wall time: 294 ms


In [52]:
train_y.head()

0    1.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: purchased, dtype: float64

# Model training

In [53]:
from lightgbm.sklearn import LGBMRanker

In [54]:
ranker = LGBMRanker(
    objective="lambdarank",
    num_leaves=200,
    metric="ndcg",
    boosting_type="dart",
    n_estimators=100,
    importance_type='gain',
    verbose=10,
)

In [55]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.983213
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.103496
[LightGBM] [Debug] init for col-wise cost 0.018170 seconds, init for row-wise cost 0.071128 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050879 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1359
[LightGBM] [Info] Number of data points in the train set: 3290121, number of used features: 19
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 18
[Ligh

In [56]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

similarity 0.9739926571807537
article_id 0.004896558101290126
postal_code 0.004252533689266868
age 0.0030786911680332754
department_no 0.0026194516114290425
product_type_no 0.002538974946260848
colour_group_code 0.0018505334252716624
section_no 0.0013562404223672112
graphical_appearance_no 0.0012260412255578397
perceived_colour_master_id 0.0011013327547253493
garment_group_no 0.0010790847264699269
perceived_colour_value_id 0.0008213211949937434
index_code 0.0003583656607447628
last_week_popularity_rank 0.0002645393188248204
index_group_no 0.00019105407774473748
FN 0.0001700231428152101
Active 0.0001168947093509754
fashion_news_frequency 6.803034666355151e-05
club_member_status 1.767229743636338e-05


# Calculate predictions

In [57]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 1.67 µs


# Create submission

In [58]:
sub = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')

In [59]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: user 2.06 s, sys: 77.7 ms, total: 2.13 s
Wall time: 2.14 s


In [60]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [61]:
sub_name = 'submission_KNN_candidates_similarity_feature'
sub.to_csv(f'{DATA_PATH}/subs/{sub_name}.csv.gz', index=False)